In [1]:
import opentrons.simulate
import json 

protocol = opentrons.simulate.get_protocol_api('2.18')
CUSTOM_LABWARE_PATH = "/Users/pozzolabadmin/Documents/codebase/experiments/peptide_phase_mapping/otto/otto/Custom_Labware/"
protocol.home()

/Users/pozzolabadmin/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/pozzolabadmin/.opentrons/deck_calibration.json not found. Loading defaults


In [2]:
# load all the labware modules
with open(CUSTOM_LABWARE_PATH+'agilent_6_reservoir_47000ul.json') as labware_file:
    reservoir_def = json.load(labware_file)
    reservoir = protocol.load_labware_from_definition(reservoir_def, location=1)

synthesized_plate = protocol.load_labware(
    load_name="corning_96_wellplate_360ul_flat",
    location=2)

tiprack_300 = protocol.load_labware(
    load_name="opentrons_96_tiprack_300ul",
    location=4)

p300 = protocol.load_instrument(
    instrument_name="p300_single_gen2",
    mount="right",
    tip_racks=[tiprack_300]
    )

In [3]:
# define the protocol
def redisperse(well, pipette):
    """ Redisperse liquid in a single well

    well : destination well (Labware.well)
    pipette : pipette module to use (InstrumentContext)
    """
    print("redispersing %s..."%well.well_name,end='\r', flush=True)
    pipette.mix(2, 30, well.bottom())
    pipette.blow_out()

    # clean the tip in a reservoir
    pipette.mix(2, 30, reservoir["A3"],rate=2)
    pipette.mix(2, 30, reservoir["A4"],rate=2)
    pipette.blow_out(reservoir["A5"])

    return

In [4]:
# redisperse full plate (iterations)
p300.pick_up_tip(tiprack_300["A1"])
for well in synthesized_plate.wells():
    redisperse(well, p300)
p300.drop_tip(tiprack_300["A1"])

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [5]:
# redisperse validation plate (iterations)
sample_rows = synthesized_plate.rows()[:3]
p300.pick_up_tip(tiprack_300["A1"])
for row in sample_rows:
    for well in row:
        redisperse(well, p300)
p300.drop_tip(tiprack_300["A1"])

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [ ]:
reservoir.wells()